# Practica 3
## Maestria en Computo Aplicado

*Topicos de la Industria I*

**Nombre:** Efren Del Real Frias  
**e-mail:** efren.delreal8824@alumnos.udg.mx  
**Fecha:** Abril 15 del 2022

# Software Requirements

* Funciones que generen trayectorias tipo Brownian Motion (BM), Correlated Random Walk (CRW) y Lefy Flight (LF).
Cada una de las funcioones deberá tomar como parámetros el numero de pasos, la velocidad y posición inicial. Además la funciones para CRW  y LF deberán tomar como parámetro el coeficiente para la distribución Cauchy. Por último, la función para LF también deberá aceptar como parámetro el exponente Levy (alpha).
La función retornará un DataFrame de Pandas con todas los puntos que forman la trayectoria.

* El dashboard deberá contener al menos los siguientes elementos (Para generar el dashboard, el estudiante podrá hacer uso de la API de su elección):
* Un panel para desplegar la trayectoria en 3D
* Un panel para desplegar la métrica de elección de la trayectoria.
* Radio Buttons (u otro tipo de selector) que permita elegir el tipo de trayectoria a análizar.
* Widgets que permitan introducir los valores enteros o de punto flotante para ajustar los parámetros de las trayectorias.
Drop down menu (u otro tipo de selector) que permita elegir la métrica a calcualr de la trayectoria bajo análisis.

# Python Header

## Modules

In [378]:
import math
import pandas as pd
import numpy as np

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable
from scipy.spatial import distance

import plotly.graph_objects as go

import panel as pn
pn.extension('plotly')

## Classes

In [379]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## Functions

### 0.- Debug purposes

In [380]:
import time
import functools

In [381]:
globalDelta: float  = 0.0

################################################################################
# measureTimeExe
################################################################################
def measureTimeExe(myFunc):

  @functools.wraps(myFunc)
  def wrapper(*args, **kwargs):
    global globalDelta

    tStart  = time.perf_counter()
    retObj  = myFunc(*args, **kwargs)
    tEnd    = time.perf_counter()

    tDelta = tEnd - tStart

    globalDelta += tDelta

    print(f"Finished {myFunc.__name__!r} in {tDelta:.4f} secs")
    return retObj

  return wrapper

In [382]:
################################################################################
# exeNTimes
################################################################################
def exeNTimes(myFunc, numTimes, *args, **kwargs):
  global globalDelta

  globalDelta = 0.0
  for _ in range(numTimes):
     retObj = myFunc(*args, **kwargs)


  print(f'Average: {globalDelta/ numTimes}')

  return retObj  

### 1.-  Brownian Motion

In [383]:
################################################################################
# trajBM_2D
################################################################################
@measureTimeExe
def trajBM_2D(n_steps=1000, speed=6, s_pos=[0, 0]):
  """
  Arguments:
        n_steps:  int         - Number of trajectory's steps
        speed:    int         - Radio's amplitud (step size)
        s_pos:    [int, int]  - Initial position (x0, y0)
  Returns:
        aTraj2D:  np.array    - BM 2D trajectory [(x0, y0), (x1, y1), ..(xn, yn)]
  """
  # Define objects
  vPos    = Vec2d(speed, 0.0)
  aTraj2D = np.empty(shape=(n_steps, 2))

  # Init Brownian Motion posicion
  # 1st step
  aTraj2D[0] = s_pos
  
  # Iterate over missing Brownian Motion steps
  for i in range(1, n_steps):

    # Gets a random direction
    theta = np.random.uniform(low=-np.pi, high=np.pi)

    # Rotates over previous position 
    vPos = vPos.rotated(theta)

    # Keeps Brownian Motion trajectory accumulating previous position 
    aTraj2D[i][0] = aTraj2D[i - 1][0] + vPos.x
    aTraj2D[i][1] = aTraj2D[i - 1][1] + vPos.y

  
  return aTraj2D

### 2.- Correlated Random Walk

In [384]:
################################################################################
# trajCRW_2D
################################################################################
@measureTimeExe
def trajCRW_2D(n_steps=1000, speed=6, s_pos=[0, 0], exp_c = 0.5):
  """
  Arguments:
        n_steps:  int         - Number of trajectory's steps
        speed:    int         - Radio's amplitud (step size)
        s_pos:    [int, int]  - Initial position (x0, y0)
        exp_c:    float       - Wrapcauchy exponent (0 < c < 1)
  Returns:
        aTraj2D:  np.array    - CRW 2D trajectory [(x0, y0), (x1, y1), ..(xn, yn)]
  """
  # Define objects
  vPos    = Vec2d(speed, 0.0)
  aTraj2D = np.empty(shape=(n_steps, 2))

  # Init CRW posicion
  # 1st step
  aTraj2D[0] = s_pos
  
  # Iterate over missing CRW steps
  for i in range(1, n_steps):

    # Gets a random direction from wrapcauchy distribution
    theta = wrapcauchy.rvs(exp_c)

    # Rotates over previous position
    vPos = vPos.rotated(theta)

    # Keeps CRW trajectory accumulating previous position 
    aTraj2D[i][0] = aTraj2D[i - 1][0] + vPos.x
    aTraj2D[i][1] = aTraj2D[i - 1][1] + vPos.y

  
  return aTraj2D

### 3.- Levy Flight

In [385]:
################################################################################
# trajLF_2D
################################################################################
@measureTimeExe
def trajLF_2D(n_steps=1000, speed=6, s_pos=[0, 0], exp_alpha = 0.7, exp_beta = 0.0, std_motion=3):
  """
  Arguments:
        n_steps:    int         - Number of trajectory's steps
        speed:      int         - Radio's amplitud (step size)
        s_pos:      [int, int]  - Initial position (x0, y0)
        exp_alpha:  float       - alpha exponent (0 < a < 2]
        exp_beta:   float       - beta exponent [0 < b < 1]
        std_motion: int         - Location
  Returns:
        aTraj2D:  np.array    - LF 2D trajectory [(x0, y0), (x1, y1), ..(xn, yn)]
  """  
  
  # Define objects
  vPos    = Vec2d(speed, 0.0)
  aTraj2D = np.empty(shape=(n_steps, 2))

  # Init posicion
  aTraj2D[0] = s_pos
  
   # Iterate over missing CRW steps
  for i in range(1, n_steps):

    # Clean object in order to make sure to do not continue in the same direction
    # or ignore a 0 step size
    theta:    float = 0.0
    step_size:  int = 0
    
    while not (bool(theta) and bool(step_size)):
      # Gets a random direction
      theta = np.random.uniform(-np.pi, np.pi)

      # Calculates how many levy's steps need to perform
      step_size = levy_stable.rvs(alpha=exp_alpha, beta=exp_beta, loc=std_motion)
      step_size = int(np.floor(step_size))

    # Rotates over previous position
    vPos = vPos.rotated(theta)

    # Keeps CRW trajectory accumulating previous position
    aTraj2D[i][0] = aTraj2D[i - 1][0] + ( vPos.x * step_size )
    aTraj2D[i][1] = aTraj2D[i - 1][1] + ( vPos.y * step_size )

  
  return aTraj2D

###  4.- Compute Path Lenght
$$
Euclidean(A, B) = \sqrt{(x_{2} - x_{1})^2  + (y_{2} - y_{1})^2}
$$

$where:$  


*   $A(x_{1}, y_{1})$
*   $B(x_{2}, y_{2})$




In [386]:
################################################################################
# compute_pathLenght
################################################################################
@measureTimeExe
def compute_pathLenght(aRW2D):
  """
  Arguments:
        aRW2D:    np.array  - Random Walk 2D size(rows = n, columns = 2)

  Returns:
        aPathLen: np.array  - Path Lenght numpy array size(rows = n, columns = 1)

  See:
        https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.euclidean.html
  """
  disEuclidean = np.array([ distance.euclidean( aRW2D[i - 1], aRW2D[i] )
                            for i in range(1, aRW2D.shape[0]) ])
  aPathLen = np.cumsum(disEuclidean)

  return aPathLen

### 5.- Mean Squared Displacement

$$
MSD = \frac{1}{N - n} \sum_{i=1}^{N-n}(r_{i+n} -r_{i})^2
$$

where $n = 1,...., N - 1$

In [387]:
################################################################################
# compute_MSD_Scalar
################################################################################
def compute_MSD_Scalar(aRW2D, tau: int) -> float:
  """
  Arguments:
        aRW2D:  np.array    - Random Walk 2D size(rows = n, columns = 2)
        tau:    integer     - Iterable number to displace two points (called "n" in above equation)
  Returns:
        msd:    float       - MSD scalar value
  """
  aDisplacement = np.array([ distance.euclidean( aRW2D[i - tau], aRW2D[i] ) 
                            for i in range(tau, aRW2D.shape[0]) ])
  
  msd: float = np.mean(np.power(aDisplacement, 2))
  
  return msd

In [388]:
@measureTimeExe
################################################################################
# compute_MSD_Array
################################################################################
def compute_MSD_Array(aRW2D):
  """
  Arguments:
        aRW2D:  np.array    - Random Walk 2D size(rows = n, columns = 2)
  Returns:
        aMSD:   np.array    - Array MSD's metrics
  """
  aMSD = np.empty(shape=(0))

  for tau in range(1, aRW2D.shape[0]):
      aMSD = np.concatenate( (aMSD, [compute_MSD_Scalar(aRW2D, tau)]), axis=0)
      
  return aMSD

## Widgets

In [389]:
wgTrajType = pn.widgets.RadioButtonGroup(
    # Core params --------------------------------------------------------------
    value       = 'BM',
    options     = ['BM', 'CRW', 'LF'],
    # Display params -----------------------------------------------------------
    name        = 'Trajectory type',
    button_type = 'default',      # [default, primary, success, warning, danger]
    width       = 200
    #margin=(25, 50, 75, 100)
    )

In [390]:
wgMtcrType = pn.widgets.Select(
    # Core params --------------------------------------------------------------
    value='PL', 
    options=['PL','MSD'],
    # Display params -----------------------------------------------------------
    name='Metrics type',
    width = 200
)

In [391]:
wgNsteps = pn.widgets.IntSlider(
    # Core params --------------------------------------------------------------
    start = 50, 
    end   = 1000, 
    step  = 50, 
    value = 200,
    # Display params -----------------------------------------------------------
    name  = 'number of steps',
    width = 200
)

In [392]:
wgSpeed = pn.widgets.IntSlider(
    # Core params --------------------------------------------------------------
    start = 1, 
    end   = 10, 
    step  = 1, 
    value = 4,
    # Display params -----------------------------------------------------------
    name  = 'Speed',
    width = 200
)

In [393]:
wgXInitPos = pn.widgets.IntInput(
    # Core params --------------------------------------------------------------
    start = 0, 
    end   = 100, 
    step  = 1, 
    value = 1,
    # Display params -----------------------------------------------------------
    name  = 'xInitPos',
    width = 90
)

In [394]:
wgYInitPos = pn.widgets.IntInput(
    # Core params --------------------------------------------------------------
    start = 0, 
    end   = 100, 
    step  = 1, 
    value = 0,
    # Display params -----------------------------------------------------------
    name  = 'yInitPos',
    width = 90
)

In [395]:
wgExpc = pn.widgets.FloatSlider(
    # Core params --------------------------------------------------------------
    start   = 0.1, 
    end     = 0.9, 
    step    = 0.1, 
    value   = 0.2,
    # Display params -----------------------------------------------------------
    name    = 'Exponent c',
    width   = 200,
    visible = False
)

In [396]:
wgExpAlpha = pn.widgets.FloatInput(
    # Core params --------------------------------------------------------------
    start = 0.1, 
    end   = 1.9, 
    step  = 0.1, 
    value = 0.7,
    # Display params -----------------------------------------------------------
    name  = 'Alpha',
    width = 90,
    visible = False
)

In [397]:
wgExpBeta = pn.widgets.FloatInput(
    # Core params --------------------------------------------------------------
    start = 0.0, 
    end   = 1.0, 
    step  = 0.1, 
    value = 0.0,
    # Display params -----------------------------------------------------------
    name  = 'Beta',
    width = 90,
    visible = False
)

In [398]:
wgLoc = pn.widgets.IntSlider(
    # Core params --------------------------------------------------------------
    start   = 0, 
    end     = 6, 
    step    = 1, 
    value   = 3,
    # Display params -----------------------------------------------------------
    name    = 'Location',
    width   = 200,
    visible = False
)

# Implementation

In [408]:
# Define Numpy array to keeps Generated Random Trajectory
aGenRW = np.empty(shape=(0))

################################################################################
# main_plotTraj
################################################################################
@pn.depends(wgTrajType, wgNsteps, wgSpeed, wgXInitPos, wgYInitPos, wgExpc, wgExpAlpha, wgExpBeta, wgLoc)
def main_plotTraj(*arg, **kwargs):
  """
  Arguments:
        arg:        tuple()     - Current widgets's
        kwars:      dict()      - Current widgets's
  Returns:
        figTraj:    go.Figure() - selected trajectory figure
  """
  # Refers to external object -  Generated Random Trajectory
  global aGenRW

  # Define Local variables
  trajType: str = arg[0]
  Nsteps:   int = arg[1]
  speedVal: int = arg[2]
  xPos:     int = arg[3]
  yPos:     int = arg[4]
  expC:   float = arg[5]
  alpha:  float = arg[6]
  beta:   float = arg[7]
  loc:      int = arg[8]
  
  plotTitle:  str = ""
  plotName:   str = ""
  
  # Calculates trajectory from the selector
  if ('BM' == trajType):
    # Disables CRW and LF params widgets
    wgExpc.visible      = False
    wgExpAlpha.visible  = False
    wgExpBeta.visible   = False
    wgLoc.visible       = False

    # Set figure params
    plotTitle = 'Brownian Motion trajectory in 3D'
    plotName = f'steps: {Nsteps}'

    # Computates and keeps Brownian Motion Trajectory
    aGenRW = trajBM_2D(n_steps=Nsteps, speed=speedVal , s_pos=[xPos, yPos])
  elif ('CRW' == trajType):
    # Disables LF params widgets and enables CRW params widgets
    wgExpc.visible      = True
    wgExpAlpha.visible  = False
    wgExpBeta.visible   = False
    wgLoc.visible       = False

    # Limit the decimals numbers
    expC = np.round(expC, 2)

     # Set figure params
    plotTitle = 'Correlated Random Walk trajectory in 3D'
    plotName = f'steps: {Nsteps}, exp_c:{expC}'

    # Computates and keeps Correlated Random Walk Trajectory
    aGenRW = trajCRW_2D(n_steps=Nsteps, speed=speedVal , s_pos=[xPos, yPos], exp_c=expC)
  elif ('LF' == trajType):
    # Disables CRW params widgets and enables LF params widgets
    wgExpc.visible      = False
    wgExpAlpha.visible  = True
    wgExpBeta.visible   = True
    wgLoc.visible       = True

    # Limit the decimals numbers
    alpha = np.round(alpha, 2)
    beta = np.round(beta, 2)

    # Set figure params
    plotTitle = 'Levy Flight trajectory in 3D'
    plotName = f'steps: {Nsteps}, alpha:{alpha}, beta:{beta}'

     # Computates and keeps Levy Flight Trajectory
    aGenRW = trajLF_2D(n_steps=Nsteps, speed=speedVal , s_pos=[xPos, yPos], exp_alpha=alpha, exp_beta=beta, std_motion=loc)


  # Init new Figure
  figTraj = go.Figure()

  # Plot trajectory in 3D space
  figTraj.add_trace(
      go.Scatter3d(
          x           = aGenRW[:,0],
          y           = aGenRW[:,1],
          z           = np.arange(len(aGenRW)),
          marker      = dict(size=2),
          line        = dict(color='blue', width=2),
          mode        = 'lines',
          name        = plotName,
          showlegend  = True
      )
  )

  figTraj.update_layout(
      title_text = plotTitle,
#      autosize=False,
#      width=800,
#      height=800,
#      scene_camera=dict(
#          up=dict(x=0, y=1,z=0),
#          center=dict(x=0, y=0, z=0),
#          eye=dict(x=0, y=0, z=1)),
      scene=dict(
          xaxis=dict( title='x_pos (m)'),
          yaxis=dict( title='y_pos (m)'),
          zaxis=dict( title='time (s)', nticks = 20)
          ) )

  return figTraj

In [406]:
################################################################################
# main_plotTraj
################################################################################
@pn.depends(wgMtcrType, wgTrajType, wgNsteps, wgSpeed, wgXInitPos, wgYInitPos, wgExpc, wgExpAlpha, wgExpBeta, wgLoc)
def main_plotMtcr(*arg, **kwargs):
  """
  Arguments:
        arg:        tuple()     - Current widgets's
        kwars:      dict()      - Current widgets's
  Returns:
        fig_mctr:   go.Figure() - selected Metrics figure
  """
  
  # Refers to external object -  Generated Random Trajectory
  global aGenRW

  # Define Local variables
  mtcrType: str = arg[0]
  metric        = np.empty(shape=(0))
  

  # Calculates metrics from the selector
  if ('PL' == mtcrType):
    metric  = compute_pathLenght(aGenRW)
  elif ('MSD' == mtcrType):
    metric  = compute_MSD_Array(aGenRW)

  # Init new Figure
  fig_mctr = go.Figure()

  # Plot trajectory
  fig_mctr.add_trace(
      go.Scatter(
          x           = np.arange(len(metric)),
          y           = metric,
          marker      = dict(size=2),
          line        = dict(width=2),
          mode        = 'lines',
          name        = f'{mtcrType}',
          showlegend  = True
  ))

  return fig_mctr

# DashBoard

In [409]:
# Dashboard
wgPanel = pn.Row(
    pn.Column(
        'Panel params',
        wgTrajType, 
        wgNsteps,
        pn.Row(wgXInitPos, wgYInitPos),
        wgSpeed,
        wgExpc,
        pn.Row(wgExpAlpha, wgExpBeta),
        wgLoc,
        wgMtcrType
        ),
    pn.Column(
        '3D Trajectory',
        main_plotTraj),
    pn.Column(
        'Metrics',
        main_plotMtcr
        )
)
wgPanel

Finished 'trajBM_2D' in 0.0008 secs
Finished 'compute_MSD_Array' in 0.1117 secs


Row
    [0] Column
        [0] Markdown(str)
        [1] RadioButtonGroup(name='Trajectory type', options=['BM', 'CRW', 'LF'], value='BM', width=200)
        [2] IntSlider(end=1000, name='number of steps', start=50, step=50, value=150, width=200)
        [3] Row
            [0] IntInput(end=100, name='xInitPos', start=0, value=1, width=90)
            [1] IntInput(end=100, name='yInitPos', start=0, width=90)
        [4] IntSlider(end=10, name='Speed', start=1, value=4, width=200)
        [5] FloatSlider(end=0.9, name='Exponent c', start=0.1, value=0.2, visible=False, width=200)
        [6] Row
            [0] FloatInput(end=1.9, name='Alpha', start=0.1, value=0.7999999999999999, visible=False, width=90)
            [1] FloatInput(end=1.0, name='Beta', start=0.0, visible=False, width=90)
        [7] IntSlider(end=6, name='Location', value=3, visible=False, width=200)
        [8] Select(name='Metrics type', options=['PL', 'MSD'], value='MSD', width=200)
    [1] Column
        [0] Markdown(str)
        [1] ParamFunction(function)
    [2] Column
        [0] Markdown(str)
        [1] ParamFunction(function)

Finished 'trajCRW_2D' in 0.0206 secs 
Finished 'compute_MSD_Array' in 0.1403 secs 
Finished 'trajCRW_2D' in 0.0145 secs 
Finished 'compute_MSD_Array' in 0.1308 secs 
Finished 'trajCRW_2D' in 0.0147 secs 
Finished 'compute_MSD_Array' in 0.1241 secs 
Finished 'trajCRW_2D' in 0.0146 secs 
Finished 'compute_MSD_Array' in 0.1311 secs 
Finished 'trajLF_2D' in 0.1720 secs 
Finished 'compute_MSD_Array' in 0.1234 secs 
Finished 'trajLF_2D' in 0.0924 secs 
Finished 'compute_MSD_Array' in 0.1246 secs 
Finished 'trajLF_2D' in 0.0925 secs 
Finished 'compute_MSD_Array' in 0.1317 secs 
Finished 'trajLF_2D' in 0.0914 secs 
Finished 'compute_MSD_Array' in 0.1289 secs 
Finished 'trajCRW_2D' in 0.0190 secs 
Finished 'compute_MSD_Array' in 0.1617 secs 
Finished 'trajCRW_2D' in 0.0147 secs 
Finished 'compute_MSD_Array' in 0.1219 secs 
Finished 'trajCRW_2D' in 0.0148 secs 
Finished 'compute_MSD_Array' in 0.1220 secs 
Finished 'trajCRW_2D' in 0.0151 secs 
Finished 'compute_MSD_Array' in 0.1356 secs 
Finished 'trajBM_2D' in 0.0021 secs 
Finished 'compute_MSD_Array' in 0.1246 secs 
Finished 'trajBM_2D' in 0.0015 secs 
Finished 'compute_MSD_Array' in 0.1224 secs 
Finished 'trajBM_2D' in 0.0019 secs 
Finished 'compute_MSD_Array' in 0.1232 secs 
Finished 'trajBM_2D' in 0.0017 secs 
Finished 'compute_MSD_Array' in 0.1338 secs